<a href="https://colab.research.google.com/github/BassamTar99/StockPrediction/blob/News_Impact_Model/News_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install newsapi-python transformers torch pandas


In [6]:
import pandas as pd
import datetime
from newsapi import NewsApiClient
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# Define the FinBERTNewsAnalyzer class
class FinBERTNewsAnalyzer:
    def __init__(self, api_key):
        self.newsapi = NewsApiClient(api_key=api_key)
        self.tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
        self.model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

    def analyze_text(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        outputs = self.model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        return {
            'positive': predictions[0][0].item(),
            'negative': predictions[0][1].item(),
            'neutral': predictions[0][2].item()
        }

    def get_stock_news(self, ticker, days=7):
        today = datetime.date.today()
        start_date = today - datetime.timedelta(days=days)

        articles = self.newsapi.get_everything(
            q=ticker,
            from_param=start_date.isoformat(),
            to=today.isoformat(),
            language='en',
            sort_by='relevancy',
            page_size=100
        )

        data = []
        for article in articles['articles']:
            headline = article['title']
            date = article['publishedAt'][:10]
            description = article.get('description', '')
            text = f"{headline}. {description}"
            sentiment_scores = self.analyze_text(text)
            dominant_sentiment = max(sentiment_scores.items(), key=lambda x: x[1])[0]

            data.append({
                'date': date,
                'headline': headline,
                'description': description,
                'url': article['url'],
                'sentiment': dominant_sentiment,
                'confidence': sentiment_scores[dominant_sentiment],
                'positive_score': sentiment_scores['positive'],
                'negative_score': sentiment_scores['negative'],
                'neutral_score': sentiment_scores['neutral']
            })

        if not data:
            print("No news found.")
            return pd.DataFrame(), pd.DataFrame()

        df = pd.DataFrame(data)

        daily = df.groupby('date').agg({
            'positive_score': 'mean',
            'negative_score': 'mean',
            'neutral_score': 'mean',
            'confidence': 'mean'
        }).reset_index()

        def get_daily_sentiment(row):
            scores = {
                'positive': row['positive_score'],
                'negative': row['negative_score'],
                'neutral': row['neutral_score']
            }
            return max(scores.items(), key=lambda x: x[1])[0]

        daily['sentiment'] = daily.apply(get_daily_sentiment, axis=1)
        return daily, df

    def get_sentiment_summary(self, ticker, days=7):
        daily_sentiment, detailed_news = self.get_stock_news(ticker, days)

        if daily_sentiment.empty:
            return "No news found for this period."

        summary = {
            'overall_sentiment': daily_sentiment['sentiment'].mode()[0],
            'average_confidence': daily_sentiment['confidence'].mean(),
            'positive_days': (daily_sentiment['sentiment'] == 'positive').sum(),
            'negative_days': (daily_sentiment['sentiment'] == 'negative').sum(),
            'neutral_days': (daily_sentiment['sentiment'] == 'neutral').sum(),
            'most_recent_news': detailed_news.sort_values('date', ascending=False).head(3)
        }

        return summary


In [7]:
# Initialize the analyzer (with your real API key)
analyzer = FinBERTNewsAnalyzer(api_key='e85a418d537146af9e862d574bd651d9')  # Replace with your actual API key


In [8]:
# Ask the user for input (stock ticker and number of days for news)
ticker = input("Enter the stock ticker (e.g., AAPL, AMZN): ").strip().upper()
days = int(input("Enter the number of days of news to analyze (e.g., 7): ").strip())


Enter the stock ticker (e.g., AAPL, AMZN): AMZN
Enter the number of days of news to analyze (e.g., 7): 3


In [9]:
# Run sentiment analysis for the user-specified stock
daily_sentiment, detailed_news = analyzer.get_stock_news(ticker, days)

# Show daily summary
print("\n📊 Daily Sentiment Summary:")
print(daily_sentiment)

# Show headline-level analysis
print("\n📰 Detailed News:")
print(detailed_news[['date', 'headline', 'sentiment', 'confidence']])

summary = analyzer.get_sentiment_summary(ticker, days)

print("\n🧠 Sentiment Summary:")
print(f"Overall Sentiment: {summary['overall_sentiment']}")
print(f"Avg. Confidence: {summary['average_confidence']:.2f}")
print(f"Positive Days: {summary['positive_days']}")
print(f"Negative Days: {summary['negative_days']}")
print(f"Neutral Days: {summary['neutral_days']}")
print("\n🗞️ Most Recent News:")
print(summary['most_recent_news'][['date', 'headline', 'sentiment']])

# Optionally, you can save the results to a CSV file
daily_sentiment.to_csv(f"{ticker}_daily_sentiment.csv", index=False)
detailed_news.to_csv(f"{ticker}_detailed_news.csv", index=False)

print(f"\nResults saved as {ticker}_daily_sentiment.csv and {ticker}_detailed_news.csv.")



📊 Daily Sentiment Summary:
         date  positive_score  negative_score  neutral_score  confidence  \
0  2025-04-22        0.359049        0.281354       0.359598    0.870460   
1  2025-04-23        0.312760        0.300970       0.386270    0.875238   
2  2025-04-24        0.459989        0.234768       0.305243    0.872625   

  sentiment  
0   neutral  
1   neutral  
2  positive  

📰 Detailed News:
          date                                           headline sentiment  \
0   2025-04-24                  What’s Happening With AMZN Stock?  negative   
1   2025-04-24        AI budgets stay hot even as tech gets weird  positive   
2   2025-04-23  Is Amazon.com, Inc. (AMZN) the Best Stock to B...   neutral   
3   2025-04-23  Is Amazon.com, Inc. (AMZN) the Best Nancy Pelo...   neutral   
4   2025-04-23  AMZN Poised to Benefit from Market Share Gains...  positive   
..         ...                                                ...       ...   
59  2025-04-22  US Stock Market Today: M

In [11]:
!pip install joblib


In [13]:
import joblib

# Save the tokenizer and model separately
joblib.dump(analyzer.tokenizer, 'finbert_tokenizer.joblib')
joblib.dump(analyzer.model, 'finbert_model.joblib')

# Optionally, save any other attributes like the API key
api_key = 'e85a418d537146af9e862d574bd651d9'  # Replace with your actual API key
joblib.dump(api_key, 'api_key.joblib')

print("Model components saved successfully.")


Model components saved successfully.
